In [ ]:
import requests, json, os
from dotenv import load_dotenv
load_dotenv()

# 1. Sgis 토큰 요청

In [ ]:
import os
import time
import requests


class SgisAPI:
    access_token = None
    token_expire = 0  # timestamp

    @classmethod
    def get_access_token(cls):
        """토큰이 만료되었으면 새로 발급받고, 아니면 캐시된 토큰을 반환"""
        if cls.access_token and time.time() < cls.token_expire:
            return cls.access_token

        AUTH_URL = "https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json"
        KOSIS_CONSUMER_KEY = os.getenv("KOSIS_CONSUMER_KEY")
        KOSIS_CONSUMER_SECRET_KEY = os.getenv("KOSIS_CONSUMER_SECRET_KEY")

        params = {
            "consumer_key": KOSIS_CONSUMER_KEY,
            "consumer_secret": KOSIS_CONSUMER_SECRET_KEY,
        }

        response = requests.get(AUTH_URL, params=params)
        data = response.json()

        if data.get("errCd") == 0:
            cls.access_token = data["result"]["accessToken"]
            cls.token_expire = time.time() + 3500  # 1시간(3600초) - 안전하게 3500초
            return cls.access_token
        else:
            raise Exception(f"❌ 토큰 발급 실패: {data}")

    @classmethod
    def request_api(cls, url: str, params: dict):
        token = cls.get_access_token()
        params["accessToken"] = token
        res = requests.get(url, params=params)
        data = res.json()


        if data.get("errCd") == -100 and "accessToken" in str(data.get("errMsg", "")).lower():
            print("⚠️ accessToken 만료됨 → 재발급 중...")
            cls.access_token = None
            token = cls.get_access_token()
            params["accessToken"] = token
            res = requests.get(url, params=params)
            data = res.json()

        return data


# 2. 주택 통계 

In [ ]:
adm_dict = {
    '종로구': '11010',
    '중구': '11020',
    '용산구': '11030',
    '성동구': '11040',
    '광진구': '11050',
    '동대문구': '11060',
    '중랑구': '11070',
    '성북구': '11080',
    '강북구': '11090',
    '도봉구': '11100',
    '노원구': '11110',
    '은평구': '11120',
    '서대문구': '11130',
    '마포구': '11140',
    '양천구': '11150',
    '강서구': '11160',
    '구로구': '11170',
    '금천구': '11180',
    '영등포구': '11190',
    '동작구': '11200',
    '관악구': '11210',
    '서초구': '11220',
    '강남구': '11230',
    '송파구': '11240',
    '강동구': '11250'
}

In [ ]:
# SGIS OpenAPI 주택통계 요청 URL
HOUSE_URL = "https://sgisapi.kostat.go.kr/OpenAPI3/stats/house.json"

params = {
    "year": "2020", # 기준연도 (2015~2023)
    "adm_cd": [adm_dict['강남구']], # 행정구역 코드 (서울특별시)
    "low_search": "0"  ,
    # 10년 이상들 조회 
     "const_year" : ["06","07","08","09","10","11"]
}
response = SgisAPI.request_api(HOUSE_URL,params)
response['result']

# 3. 연령층 분포

In [27]:
import os,requests,json
from dotenv import load_dotenv
load_dotenv()
import requests
import urllib.parse


GONG_GONG_DATA_API_KEY = os.getenv("GONG_GONG_DATA_API_KEY")


# 3️⃣ 기본 요청 URL
url = f"https://apis.data.go.kr/1741000/admmSexdAgePpltn/selectAdmmSexdAgePpltn"
params = {
    "serviceKey": "FcTAw%2BPnCRMmJ0VuH0Yj37qIgrXFMN%2BHd0BNnLlKUPtbGqhAlkhR%2FdzVPwr79x%2BzV%2FmeX3Ld7Olw7jlWpQs%2Fmw%3D%3D",
    "stdYm": "202409",         # 기준연월
    "admCd": "1111054000",     # 행정구역 코드 (예: 서울 강남구 역삼1동)
    "type": "json",            # 응답형식
    # "numOfRows": 10,
    # "pageNo": 1
}

# 4️⃣ API 호출
response = requests.get(url, params=params)

# 5️⃣ 응답 확인
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("오류:", response.status_code, response.text)


오류: 401 Unauthorized



In [22]:
GONG_GONG_DATA_API_KEY

'FcTAw%2BPnCRMmJ0VuH0Yj37qIgrXFMN%2BHd0BNnLlKUPtbGqhAlkhR%2FdzVPwr79x%2BzV%2FmeX3Ld7Olw7jlWpQs%2Fmw%3D%3D'

In [24]:


import requests

url = "https://apis.data.go.kr/1741000/admmSexdAgePpltn/selectAdmmSexdAgePpltn"
params = {
    "serviceKey": GONG_GONG_DATA_API_KEY,  # ✅ 인코딩키 사용!
    "stdYm": "202409",
    "admCd": "1168010100",
    "type": "json"
}

response = requests.get(url, params=params)
print(response)

<Response [401]>
